In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt 
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from tqdm import tqdm_notebook as tqdm
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
    
import h5py
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
try:
    import dill as pickle
except ImportError:
    import pickle

# Loading Precomputed Data

In [4]:
# Loading Image Features
train_features = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

Photos: train=6000


In [5]:
# Load Train Descriptions, which contains, 5 captions corresponding to a key
train_descriptions = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/train_descriptions.pkl", "rb"))
print('Photos: train=%d' % len(train_descriptions))

Photos: train=6000


In [6]:
# Loading embedding_matrix
embedding_matrix = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/Embedding_Matrix.pkl", "rb"))
print('Embedding Matrix shape = (%d, %d)' % (embedding_matrix.shape))

Embedding Matrix shape = (2064, 200)


In [7]:
# Loading wordtoix
wordtoix = load(open("/home/vinit/Desktop/Projects/Image Captioning/Dataset/wordtoix.pkl", "rb"))
print('Wordtoix Length = %d' % len(wordtoix))

Wordtoix Length = 2063


# Data Generator

In [17]:
# Used in a call to model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            
            photo = photos[key]
            for desc in desc_list:
                
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    

In [9]:
train_features

{'2513260012_03d33305cf.jpg': array([0.08447918, 0.09467561, 0.09229978, ..., 0.07690574, 1.456184  ,
        0.45119023], dtype=float32),
 '2903617548_d3e38d7f88.jpg': array([0.17602412, 0.5156974 , 0.4131565 , ..., 0.651136  , 0.35627756,
        0.97767144], dtype=float32),
 '3338291921_fe7ae0c8f8.jpg': array([0.16187225, 0.13512476, 0.04707851, ..., 0.35250106, 0.25436395,
        0.38996473], dtype=float32),
 '488416045_1c6d903fe0.jpg': array([0.45007303, 0.07103869, 0.14107409, ..., 0.5103904 , 1.1378491 ,
        0.473709  ], dtype=float32),
 '2644326817_8f45080b87.jpg': array([0.26103917, 0.8520201 , 0.03016772, ..., 0.05509172, 1.0150243 ,
        0.9370173 ], dtype=float32),
 '218342358_1755a9cce1.jpg': array([0.7362431 , 0.1655053 , 0.12633784, ..., 0.32769364, 0.28599402,
        0.5855335 ], dtype=float32),
 '2501968935_02f2cd8079.jpg': array([0.15158759, 0.0629065 , 1.0291252 , ..., 0.07256524, 1.0983651 ,
        0.997336  ], dtype=float32),
 '2699342860_5288e203ea.jpg':

# Defining the Model

In [18]:
max_length = 35 
vocab_size = embedding_matrix.shape[0]
embedding_dim = embedding_matrix.shape[1]

In [19]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 35)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 35, 200)      412800      input_4[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           input_3[0][0]                    
__________________________________________________________________________________________________
dropout_3 

In [21]:
model.layers[2]

In [22]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [24]:
model.optimizer.lr = 0.0001
epochs = 20
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

In [ ]:
!pip3 install h5py
import h5py

  Using cached https://files.pythonhosted.org/packages/30/99/d7d4fbf2d02bb30fb76179911a250074b55b852d34e98dd452a9f394ac06/h5py-2.9.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/fb/00a976f728d0d1fecfe898238ce23f502a721c0ac0ecfedb80e0d88c64e9/six-1.12.0-py2.py3-none-any.whl


In [1]:
model.save('xyz.h5py')

NameError: name 'model' is not defined

In [29]:
state =cell.zero_state(batchsize, tf.float32).eval()

NameError: name 'cell' is not defined

In [1]:
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.h5')

NameError: name 'tqdm' is not defined

In [117]:
with open('/home/vinit/Desktop/Projects/Image Captioning/Dataset/Caption_generator_Model.pkl', 'wb') as model:
    pickle.dump(model, model)
    

TypeError: cannot serialize '_io.BufferedWriter' object

In [132]:
model.save('my_model_01.h5py')

AttributeError: '_io.BufferedWriter' object has no attribute 'save'

In [127]:
from keras.callbacks import ModelCheckpoint as Save

In [129]:
from keras.models import load_model
from keras.models

In [25]:

model.save('./model_weights/my_model.h5')  # creates a HDF5 file 'my_model.h5'

AttributeError: 'float' object has no attribute 'eval'

In [ ]:
model.save('myModel.h5py')

In [ ]:
with open('./model_weights/TEST.pkl', )